In [1]:
using LinearAlgebra, CSV, Random, Tables, COSMO

In [2]:
data = CSV.File("../NC-Data.csv") |> Tables.matrix;
# Matrix sqrt: via diagonalization and sqrt eigenvalues
raw_data = sqrt(data)';
n = size(data, 1);
N = n * n;
k = 7;
# All close.
maximum(abs.(raw_data * raw_data .- data))

4.884981308350689e-15

In [3]:
nodes = CSV.File("../NC-K7-Trace-Nodes.csv") |> Tables.matrix;
bounds = CSV.File("../NC-K7-Trace-Bounds.csv") |> Tables.matrix;

In [26]:
# node_id = 50
node_id = 300
y = nodes[:, node_id]
sum(nodes[:, node_id] .== 1)

3

In [5]:
bounds[:, node_id]

3-element Array{Float64,1}:
 3.608137513865839
 6.26741838326191
 7.0

In [6]:
selected_data = findall(nodes[:, node_id] .== 1)

3-element Array{Int64,1}:
 58
 60
 62

In [7]:
s = maximum(svd(raw_data[:, selected_data]).S)
u = svd(raw_data[:, selected_data]).U[:, 1];
v = svd(raw_data[:, selected_data]).V[:, 1];
D = Matrix(1.0I, 101, 101)[:, selected_data];
s

1.6649665589270974

In [8]:
rank_one_term = (raw_data * u) .^ 2
rank_one_term[nodes[:, node_id] .== 0] .= 0
rank_one_term

101-element Array{Float64,1}:
 0.042608386423828526
 0.12169366148453958
 0.0010827657107646796
 0.0
 0.3576958952379641
 0.590931332887428
 0.0028134344289715506
 0.045710787852978954
 0.0134758787384476
 0.0402507143194045
 ⋮
 0.19651099861244686
 0.03521501895354107
 0.00462342769518394
 0.026081129247667985
 0.0008624637095038933
 0.37524469637807883
 0.11466762911354514
 0.03909959370980699
 0.0

In [9]:
# Upper bound on the contribution from adding one variable into ||E||^2.
# The upper bound comes from ||E||_F^2
# upper bounded by ||(I-uu')MD||_F^2. Also, we know that normCommunities
# has column norm of 1 for every variable.
# We should add another upper bound based on taking k columns from this
# row into ||E||_F.
residual_term = 1 .- rank_one_term

101-element Array{Float64,1}:
 0.9573916135761715
 0.8783063385154604
 0.9989172342892353
 1.0
 0.6423041047620359
 0.40906866711257195
 0.9971865655710285
 0.954289212147021
 0.9865241212615524
 0.9597492856805955
 ⋮
 0.8034890013875531
 0.9647849810464589
 0.995376572304816
 0.973918870752332
 0.9991375362904961
 0.6247553036219211
 0.8853323708864549
 0.960900406290193
 1.0

In [10]:
lb_proj_nodes = rank_one_term .>= sort(rank_one_term)[end-k+1]
lb_proj = sum(rank_one_term[lb_proj_nodes])
lb_diag = maximum(svd(raw_data[:, lb_proj_nodes]).S) ^ 2

6.070332386128684

In [11]:
bounds[:, node_id]

3-element Array{Float64,1}:
 3.608137513865839
 6.26741838326191
 7.0

In [12]:
lambda_1 = bounds[2, node_id]
lambda_2 = bounds[3, node_id] - lb_diag
lambda_2_lb = bounds[3, node_id] - lambda_1
lambda_2

0.9296676138713158

In [28]:
model = COSMO.Model()

# Number of auxiliary helper variables
# v[n+1] -> sqrt(x(1-x))
aux = n+1

ones_c = COSMO.Constraint(
    [Vector{Float64}(y .== 1)' 0],
    [-Float64(sum(y .== 1))],
    COSMO.ZeroSet)

zeros_c = COSMO.Constraint(
    [Vector{Float64}(y .== 0)' 0],
    [0.],
    COSMO.ZeroSet)

nonnegative_c = COSMO.Constraint(
    Matrix{Float64}(I, aux, aux),
    zeros(aux),
    COSMO.Nonnegatives)

box_c = COSMO.Constraint(
    [I zeros(n, 1)],
    zeros(n),
    COSMO.Box(zeros(n), ones(n)))

trace_c = COSMO.Constraint(
    [ones(1, n) 0.],
    [-Float64(k)],
    COSMO.ZeroSet)

x_e = residual_term' / lambda_1^2
# sqrt(x(1-x)) >= x_3
power_cone = [
    x_e 0;
    -x_e 0;
    zeros(1, n) 1
]
power_cone = COSMO.Constraint(
    power_cone,
    [0.; 1; 0],
    COSMO.PowerCone(0.5))

if (false)
    assemble!(
        model,
        zeros(aux, aux),
        [-(x_e .* lambda_2) -sqrt(lambda_1 * lambda_2)],
        [ones_c, zeros_c, nonnegative_c, box_c, trace_c, power_cone])
else
    assemble!(
        model,
        zeros(aux, aux),
        [-(x_e .* lambda_1 + rank_one_term') -sqrt(lambda_1 * lambda_2)],
        [ones_c, zeros_c, nonnegative_c, box_c, trace_c, power_cone])
end

In [29]:
result = COSMO.optimize!(model)

>>> COSMO - Results
Status: 

Solved
Iterations: 50 (incl. 0 safeguarding iterations)
Optimal Objective: -6.519
Runtime: 1.16ms
Setup Time: 0.28ms

Avg Iter Time: 0.02ms

In [30]:
result.x

102-element Array{Float64,1}:
  5.110740722057198e-12
  8.333235538660357e-13
  7.349870555992568e-12
 -3.384617551620242e-15
 -1.2405615912363124e-11
  0.9999999999758519
  7.2564407512562125e-12
  4.936161535943614e-12
  6.692023616728419e-12
  5.235117941419649e-12
  ⋮
  5.508264952263753e-12
  7.169769774529623e-12
  5.998792767623775e-12
  7.386520937691893e-12
 -1.3441839834005114e-11
  1.2006195741140334e-12
  5.283954887870088e-12
 -3.384617551620242e-15
  0.15989526443099525

In [31]:
maximum(svd(raw_data .* result.x[1:n]).S .^ 2)

6.070332385850657

In [32]:
maximum(svd((raw_data * (I - u * u')) .* result.x[1:n]).S .^ 2)

0.7302230035007676

In [33]:
sqrt(residual_term' * result.x[1:n])

1.0155512884636682